# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [708]:
def calcular_valor(numero, palo):
    valores = {
        (1, 'espada'): 14,
        (1, 'basto'): 13,
        (7, 'oro'): 12,
        (7, 'copa'): 11,
        3: 10,
        2: 9,
        1: 8,
        12: 7,
        11: 6,
        10: 5,
        7: 4,
        6: 3,
        5: 2,
        4: 1
    }
    return valores.get((numero, palo), valores.get(numero, 0))


class Carta:
    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo

    @property
    def valor(self):
        return calcular_valor(self.numero, self.palo)
    def __str__(self):
        return f"{self.numero:2}"
    def __repr__(self):
        return self.__str__()

    def __gt__(self, otra):
        return self.valor > otra.valor

c = Carta(1, "oro")
print (c)
c = Carta(1, "espada")
print (c)
print(c.valor)




 1
 1
14


**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [697]:
import random

numeros = [1, 2, 3, 4, 5, 6, 7, 10, 11, 12]
palos = ['oro', 'copa', 'espada', 'basto']

class Mazo:
    def __init__(self):
        self.cartas = [(numero, palo) for numero in numeros for palo in palos] 
        self.mezclar()       

    def mezclar(self):
        random.shuffle(self.cartas)
    
    def repartir(self, cantidad):
        return [self.cartas.pop() for _ in range(cantidad)]

m = Mazo()
m.mezclar()
print(m.cartas)
print(m.repartir(3))


[(3, 'oro'), (4, 'basto'), (3, 'espada'), (6, 'copa'), (11, 'espada'), (2, 'copa'), (12, 'espada'), (2, 'oro'), (2, 'basto'), (6, 'basto'), (6, 'oro'), (12, 'basto'), (10, 'copa'), (6, 'espada'), (7, 'espada'), (5, 'copa'), (7, 'copa'), (10, 'oro'), (3, 'copa'), (10, 'basto'), (10, 'espada'), (3, 'basto'), (1, 'espada'), (5, 'oro'), (7, 'oro'), (1, 'copa'), (1, 'basto'), (5, 'espada'), (4, 'oro'), (7, 'basto'), (2, 'espada'), (4, 'copa'), (1, 'oro'), (12, 'copa'), (4, 'espada'), (11, 'copa'), (12, 'oro'), (11, 'basto'), (5, 'basto'), (11, 'oro')]
[(11, 'oro'), (5, 'basto'), (11, 'basto')]


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [721]:
class Jugador:
    
    def __init__(self, nombre):
        self.nombre = nombre
        self.cartas = []
        self.puntos = 0 

    def recibir_cartas(self, cartas):
        self.cartas = cartas
    
    def jugar_carta(self):
        random.shuffle(self.cartas)
        carta_jugada=self.cartas.pop()
        return carta_jugada
    def resultado(self, carta):
        numero, palo = carta
        return calcular_valor(numero, palo)
        
    def __str__(self):
        return self.nombre
    def __repr__(self) -> str:
        return self.__str__()
m = Mazo()  # Asumiendo que ya tienes la clase Mazo definida

jugador = Jugador("Juan")
jugador.recibir_cartas(m.repartir(3))

print("recibe las cartas:",jugador.cartas)
carta_jugada = jugador.jugar_carta()
print("carta jugada:",carta_jugada)
print("cartas q le quedaron:",jugador.cartas)
print("resultado:",jugador.resultado(carta_jugada))

recibe las cartas: [(1, 'copa'), (3, 'espada'), (12, 'copa')]
carta jugada: (12, 'copa')
cartas q le quedaron: [(1, 'copa'), (3, 'espada')]
resultado: 7


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se terminen las cartas del mazo.

In [723]:
class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.mano = []
        self.puntos = 0

    def recibir_cartas(self, cartas):
        self.mano.extend(cartas)

    def jugar_carta(self):
        return self.mano.pop(random.randint(0, len(self.mano) - 1))

    def __str__(self):
        return self.nombre

    def __repr__(self):
        return self.__str__()

class Partida:
    def __init__(self, jugador1, jugador2):
        self.jugadores = [jugador1, jugador2]
        self.mazo = Mazo()

    def jugar_ronda(self):
        return [[j.jugar_carta() for j in self.jugadores] for _ in range(3)]
    
    def evaluar_jugada(self, jugadas):
        ganadas = [sum(1 for c1, c2 in jugadas if c1 > c2),
                   sum(1 for c1, c2 in jugadas if c2 > c1)]
        
        for g, jugador in zip(ganadas, self.jugadores):
            if g >= 2:
                jugador.puntos += 1
                print(f"{jugador} gana la ronda")

    def puntajes(self):
        print(f"\nPuntos:")
        for jugador in self.jugadores:
            print(f" - {jugador}: {jugador.puntos}")
        print('-' * 20)

    def jugar(self):
        ronda = 0
        while all(j.puntos < 15 for j in self.jugadores):
            self.mazo = Mazo()
            ronda += 1
            print(f"Ronda {ronda}: {' vs '.join(str(j) for j in self.jugadores)}")

            for jugador in self.jugadores:
                jugador.recibir_cartas(self.mazo.repartir(3))
            
            jugadas = self.jugar_ronda()
            for jugada in jugadas:
                print(' ', jugada)
            self.evaluar_jugada(jugadas)
            
            self.puntajes()

        for jugador in self.jugadores:
            if jugador.puntos >= 15:
                print(f"{jugador} gana la partida")

# Ejemplo de uso
p = Partida(Jugador("👨🏻‍🎓 Carlos"), Jugador("👩🏻‍🔧 Analia"))
p.jugar()

Ronda 1: 👨🏻‍🎓 Carlos vs 👩🏻‍🔧 Analia
  [(2, 'copa'), (2, 'oro')]
  [(11, 'copa'), (10, 'basto')]
  [(6, 'basto'), (7, 'basto')]
👩🏻‍🔧 Analia gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 0
 - 👩🏻‍🔧 Analia: 1
--------------------
Ronda 2: 👨🏻‍🎓 Carlos vs 👩🏻‍🔧 Analia
  [(7, 'copa'), (5, 'oro')]
  [(1, 'copa'), (6, 'espada')]
  [(11, 'copa'), (1, 'oro')]
👨🏻‍🎓 Carlos gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 1
 - 👩🏻‍🔧 Analia: 1
--------------------
Ronda 3: 👨🏻‍🎓 Carlos vs 👩🏻‍🔧 Analia
  [(10, 'espada'), (12, 'oro')]
  [(11, 'oro'), (1, 'espada')]
  [(11, 'copa'), (10, 'oro')]
👨🏻‍🎓 Carlos gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 2
 - 👩🏻‍🔧 Analia: 1
--------------------
Ronda 4: 👨🏻‍🎓 Carlos vs 👩🏻‍🔧 Analia
  [(2, 'espada'), (7, 'espada')]
  [(5, 'copa'), (6, 'espada')]
  [(4, 'copa'), (4, 'espada')]
👩🏻‍🔧 Analia gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 2
 - 👩🏻‍🔧 Analia: 2
--------------------
Ronda 5: 👨🏻‍🎓 Carlos vs 👩🏻‍🔧 Analia
  [(5, 'copa'), (2, 'basto')]
  [(7, 'espada'), (4, 'oro')]
  [(12, 'oro'), (7, 'copa